In [1]:
import cv2
import pandas as pd
import numpy as np

from tqdm import tqdm


#import segmentation_models_pytorch as smp
import matplotlib.pyplot as plt
from IPython.display import clear_output 
import time
import os
import json
import random

random_seed = 42
np.random.seed(random_seed)
random.seed(random_seed)

In [2]:
# RLE 디코딩 함수
def rle_decode(mask_rle, shape):
    if mask_rle == -1:
        return np.zeros(shape, dtype=np.uint8)
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

# RLE 인코딩 함수
def rle_encode(mask):
    if np.sum(mask) == 0:
        return '-1'
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [3]:
csv_files = []
csv_files.append(pd.read_csv('../submit/b5_jhsstride_040_yjypost.csv'))  # CSV 파일 경로
csv_files.append(pd.read_csv('../submit/submit_deepsplit_post.csv'))

csv_files.append(pd.read_csv('../submit/b4_48batch_27000steps_040_yjypost.csv'))


csv_files.append(pd.read_csv('../submit/deepexc_a15_post.csv'))
csv_files.append(pd.read_csv('../submit/b5_cityscapes_040_42000steps_yjypost.csv')) #b5 48btch 42000steps
csv_files.append(pd.read_csv('../submit/b5_ade_040_36000steps_yjypost.csv')) #b5 48btch 36000steps
### 재현불가능한 csv파일 ###

#csv_files.append(pd.read_csv('../submit/before42seed/b5_040_pre_yjypost.csv'))
#csv_files.append(pd.read_csv('../submit/before42seed/nvidiamit_b5_040_yjypost.csv'))
#csv_files.append(pd.read_csv('../submit/rubbish/b5_040_preprocessed_yjypost.csv'))

###



# 결과를 저장할 새로운 DataFrame 생성
result_data = pd.DataFrame(columns=['img_id', 'mask_rle'])

hflencsv=len(csv_files) // 2

for i in tqdm(range(60640)):
    sumask=np.zeros((224,224), dtype=np.uint8)
    for csv_file in csv_files:
        mask_data = csv_file['mask_rle'][i]
        mask = rle_decode(mask_data, (224,224))
        sumask += mask
    sumask=np.where(sumask > hflencsv,1,0)

    result_data.loc[i] = [csv_file['img_id'][i], rle_encode(sumask)]
print(result_data.head)


100%|██████████| 60640/60640 [04:29<00:00, 225.39it/s]

<bound method NDFrame.head of            img_id                                           mask_rle
0      TEST_00000  20002 9 20226 10 20258 3 20450 11 20480 8 2067...
1      TEST_00001  35838 3 36060 5 36282 7 36505 8 36727 10 36950...
2      TEST_00002                                                 -1
3      TEST_00003  21 25 73 12 243 27 297 12 467 27 521 11 690 29...
4      TEST_00004  16891 20 17114 24 17148 11 17338 46 17561 47 1...
...           ...                                                ...
60635  TEST_60635                                                 -1
60636  TEST_60636  29094 4 29316 10 29538 13 29760 15 29982 17 30...
60637  TEST_60637  58 14 88 17 282 14 312 17 506 15 536 18 729 17...
60638  TEST_60638  14486 2 14703 14 14923 19 15146 20 15369 22 15...
60639  TEST_60639  11454 2 11673 9 11890 17 12111 20 12333 22 125...

[60640 rows x 2 columns]>


In [4]:
result_data.to_csv('../submit/ensemble/ensemble9.csv', index=False)

In [5]:
print(len(csv_files))

7
